In [1]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Importing data

In [2]:
df = pd.read_csv('clean_data.csv')
# df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,...,city_name,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description
0,2015-01-01 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,Madrid,-5.825,0.971,63,1,309,0,800,clear,sky is clear
1,2015-01-01 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,Madrid,-5.825,0.971,63,1,309,0,800,clear,sky is clear
2,2015-01-01 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,Madrid,-6.964,0.971,64,1,273,0,800,clear,sky is clear
3,2015-01-01 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,Madrid,-6.964,0.971,64,1,273,0,800,clear,sky is clear
4,2015-01-01 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,Madrid,-6.964,0.971,64,1,273,0,800,clear,sky is clear


In [3]:
df.columns

Index(['time', 'generation_biomass', 'generation_fossil_brown_coal/lignite',
       'generation_fossil_gas', 'generation_fossil_hard_coal',
       'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption',
       'generation_hydro_run-of-river_and_poundage',
       'generation_hydro_water_reservoir', 'generation_nuclear',
       'generation_other', 'generation_other_renewable', 'generation_solar',
       'generation_waste', 'generation_wind_onshore', 'total_load_actual',
       'price_day_ahead', 'price_actual', 'city_name', 'temp', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description'],
      dtype='object')

Need to X-y-split AND train-test-split BEFORE I apply transformations, 
then train transformation on training set only

In [4]:
df = df.sample(frac=1)

In [5]:
y = df['price_actual']
X = df.drop(['price_actual','time', 'city_name'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Dividing into numericals and categoricals

In [6]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(object)

X_test_num = X_test.select_dtypes(np.number)
X_test_cat = X_test.select_dtypes(object)

In [7]:
X_train_num.head()

,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,generation_wind_onshore,total_load_actual,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id
430,472.0,545.0,4067.0,3954.0,268.0,3063.0,977.0,518.0,7102.0,82.0,...,7103.0,23397.0,31.17,1.899,0.944,97,3,23,68,500
32374,345.0,0.0,6087.0,4058.0,218.0,1.0,1123.0,2843.0,6050.0,59.0,...,6489.0,30560.0,66.69,35.950,1.020,14,1,0,0,800
22584,383.0,726.0,9351.0,5230.0,331.0,0.0,580.0,568.0,6315.0,15.0,...,5699.0,33547.0,45.97,33.570,1.016,17,5,210,0,800
21631,354.0,878.0,8319.0,6657.0,302.0,169.0,741.0,536.0,5879.0,50.0,...,2340.0,27596.0,51.57,18.000,1.016,77,1,10,40,802
6037,571.0,838.0,6009.0,7191.0,317.0,0.0,622.0,4053.0,4952.0,83.0,...,1008.0,28461.0,60.50,19.890,1.015,34,5,250,40,802


#### Normalizing the numerical data

In [8]:
from sklearn.preprocessing import MinMaxScaler
def EscaladorMinMax(data, MinMaxtransformer):
    X_normalized = MinMaxtransformer.transform(data)
    print(X_normalized.shape)
    X_normalized = pd.DataFrame(X_normalized,columns=data.columns)
    return X_normalized

In [9]:
MinMaxtransformer = MinMaxScaler().fit(X_train_num) # Only run once

In [10]:
X_train_num_escalado = EscaladorMinMax(X_train_num, MinMaxtransformer)
X_train_num_escalado.head()

(28014, 23)


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,generation_wind_onshore,total_load_actual,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id
0,0.797297,0.545546,0.203005,0.473023,0.596882,0.677205,0.489724,0.053248,0.997892,0.773585,...,0.407376,0.233133,0.291304,0.221899,0.147826,0.97,0.166667,0.063889,0.68,0.496689
1,0.582770,0.000000,0.303833,0.485465,0.485523,0.000221,0.562907,0.292249,0.850077,0.556604,...,0.372161,0.544920,0.646753,0.914021,0.808696,0.14,0.055556,0.000000,0.00,0.993377
2,0.646959,0.726727,0.466757,0.625673,0.737194,0.000000,0.290727,0.058388,0.887312,0.141509,...,0.326852,0.674937,0.439408,0.865645,0.773913,0.17,0.277778,0.583333,0.00,0.993377
3,0.597973,0.878879,0.415244,0.796387,0.672606,0.037365,0.371429,0.055099,0.826050,0.471698,...,0.134205,0.415905,0.495447,0.549169,0.773913,0.77,0.055556,0.027778,0.40,0.996689
4,0.964527,0.838839,0.299940,0.860270,0.706013,0.000000,0.311779,0.416632,0.695799,0.783019,...,0.057811,0.453556,0.584809,0.587585,0.765217,0.34,0.277778,0.694444,0.40,0.996689


In [11]:
X_test_num_escalado = EscaladorMinMax(X_test_num, MinMaxtransformer)
X_test_num_escalado.head()

(7004, 23)


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,generation_wind_onshore,total_load_actual,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id
0,0.329392,0.000000,0.158331,0.116521,0.496659,0.378289,0.824060,0.416118,0.997049,0.500000,...,0.413168,0.173283,0.069449,0.304728,0.295652,0.76,0.055556,0.136111,0.0,0.993377
1,0.535473,0.446446,0.235250,0.609882,0.556793,0.026531,0.484211,0.089638,0.997330,0.113208,...,0.180775,0.249064,0.456319,0.268669,0.921739,0.70,0.055556,0.000000,0.0,0.993377
2,0.554054,0.649650,0.531247,0.661323,0.712695,0.000442,0.323308,0.387850,0.984404,0.641509,...,0.032003,0.493776,0.726609,0.772552,0.808696,0.28,0.055556,0.388889,0.2,0.531457
3,0.547297,0.904905,0.447839,0.710133,0.688196,0.000000,0.297243,0.285876,0.982155,0.481132,...,0.155712,0.566815,0.605624,0.457661,0.243478,1.00,0.055556,0.205556,0.8,0.496689
4,0.506757,0.000000,0.245932,0.162579,0.592428,0.364360,0.541855,0.099507,0.853309,0.556604,...,0.443221,0.230957,0.479936,0.378430,0.843478,0.81,0.055556,0.111111,0.2,0.895695


#### Onehot encoding for the categorical variables

In [12]:
X_train_cat.head()

,weather_main,weather_description
430,rain,light rain
32374,clear,sky is clear
22584,clear,sky is clear
21631,clouds,scattered clouds
6037,clouds,scattered clouds


In [13]:
# X_train_cat['weather_description'].unique()

ValueError: Found unknown categories ['thunderstorm with heavy rain'] in column 1 during transform

In [14]:
from sklearn.preprocessing import OneHotEncoder
def OneHotEncoding(data, OneHotEncoder):
    # print(OneHotEncoder.categories_)
    encoded = OneHotEncoder.transform(data).toarray()
    # print(encoded)
    onehot_encoded = pd.DataFrame(encoded)
    return onehot_encoded

In [15]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore').fit(X_train_cat) # Only run once
# Ahora habría que llamar a la función

In [16]:
X_train_cat['weather_description'].unique()

array(['light rain', 'sky is clear', 'scattered clouds', 'few clouds',
       'overcast clouds', 'broken clouds', 'light intensity shower rain',
       'moderate rain', 'mist', 'shower rain', 'fog', 'snow',
       'proximity thunderstorm', 'light intensity drizzle', 'drizzle',
       'proximity shower rain', 'sleet', 'thunderstorm',
       'light rain and snow', 'heavy intensity drizzle',
       'heavy intensity rain', 'light snow',
       'light intensity drizzle rain', 'haze',
       'heavy intensity shower rain', 'thunderstorm with light rain',
       'thunderstorm with rain', 'rain and drizzle', 'heavy snow',
       'rain and snow', 'light thunderstorm',
       'thunderstorm with heavy rain'], dtype=object)

In [17]:
X_test_cat['weather_description'].unique()

array(['sky is clear', 'proximity shower rain', 'light rain', 'fog',
       'shower rain', 'broken clouds', 'scattered clouds', 'drizzle',
       'overcast clouds', 'few clouds', 'mist',
       'light intensity shower rain', 'moderate rain',
       'light intensity drizzle', 'heavy intensity shower rain',
       'proximity thunderstorm', 'light snow', 'snow',
       'heavy intensity drizzle', 'thunderstorm', 'heavy intensity rain',
       'thunderstorm with rain', 'thunderstorm with light rain',
       'light rain and snow', 'sleet', 'rain and snow', 'haze'],
      dtype=object)

In [18]:
X_train_cat_encoded = OneHotEncoding(X_train_cat, encoder)
X_test_cat_encoded = OneHotEncoding(X_test_cat, encoder)
X_train_cat_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X_test_cat_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X_train = pd.concat([X_train_num_escalado, X_train_cat_encoded], axis=1).reset_index(drop=True)
X_test = pd.concat([X_test_num_escalado, X_test_cat_encoded], axis=1).reset_index(drop=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [21]:
X_train.isna().sum()

generation_biomass                      0
generation_fossil_brown_coal/lignite    0
generation_fossil_gas                   0
generation_fossil_hard_coal             0
generation_fossil_oil                   0
                                       ..
34                                      0
35                                      0
36                                      0
37                                      0
38                                      0
Length: 62, dtype: int64

#### Building a Linear Regression Model

In [22]:
from sklearn import linear_model
linear_model = linear_model.LinearRegression()

In [23]:
X_train.shape, y.shape

((28014, 62), (35018,))

In [24]:
linear_model.fit(X_train,y_train)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [25]:
from sklearn.metrics import r2_score
predictions = linear_model.predict(X_train)
r2_score(y_train, predictions)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.5920610087520537

In [26]:
predictions_test = linear_model.predict(X_test)
r2_score(y_test, predictions_test)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.5919321735305867

In [27]:
y_test[:5]

0    57.19
1    51.82
2    79.53
3    69.27
4    55.50
Name: price_actual, dtype: float64

In [28]:
predictions_test[:5]

array([29.734375  , 51.1875    , 75.75585938, 66.91796875, 60.765625  ])